In [ ]:
#| default_exp components

# Components

In [ ]:
#| export
from dataclasses import dataclass, asdict, is_dataclass, make_dataclass, replace, astuple, MISSING
from bs4 import BeautifulSoup, Comment

from fastcore.utils import *
from fastcore.xml import *
from fastcore.meta import use_kwargs, delegates

import types

try: from IPython import display
except ImportError: display=None

In [ ]:
from lxml import html as lx
from pprint import pprint

In [ ]:
#| export
def show(ft,*rest):
    if rest: ft = (ft,)+rest
    return display.HTML(to_xml(ft))

In [ ]:
#| export
voids = set('area base br col command embed hr img input keygen link meta param source track wbr !doctype'.split())
named = set('a button form frame iframe img input map meta object param select textarea'.split())
html_attrs = 'id cls title style accesskey contenteditable dir draggable enterkeyhint hidden inert inputmode lang popover spellcheck tabindex translate'.split()
hx_attrs = 'get post put delete patch trigger target swap include select indicator push_url confirm disable replace_url on'
hx_attrs = html_attrs + [f'hx_{o}' for o in hx_attrs.split()]

In [ ]:
#| export
def xt_html(tag: str, *c, id=None, cls=None, title=None, style=None, **kwargs):
    if len(c)==1 and isinstance(c[0], (types.GeneratorType, map, filter)): c = tuple(c[0])
    kwargs['id'],kwargs['cls'],kwargs['title'],kwargs['style'] = id,cls,title,style
    tag,c,kw = ft(tag, *c, **kwargs)
    if tag in named and 'id' in kw and 'name' not in kw: kw['name'] = kw['id']
    return FT(tag,c,kw, void_=tag in voids)

In [ ]:
#| export
@use_kwargs(hx_attrs, keep=True)
def xt_hx(tag: str, *c, target_id=None, **kwargs):
    if target_id: kwargs['hx_target'] = '#'+target_id
    return xt_html(tag, *c, **kwargs)

In [ ]:
#| export
_g = globals()
_all_ = [
    'A', 'Abbr', 'Address', 'Area', 'Article', 'Aside', 'Audio', 'B', 'Base', 'Bdi', 'Bdo', 'Blockquote', 'Body', 'Br',
    'Button', 'Canvas', 'Caption', 'Cite', 'Code', 'Col', 'Colgroup', 'Data', 'Datalist', 'Dd', 'Del', 'Details', 'Dfn',
    'Dialog', 'Div', 'Dl', 'Dt', 'Em', 'Embed', 'Fencedframe', 'Fieldset', 'Figcaption', 'Figure', 'Footer', 'Form',
    'H1', 'Head', 'Header', 'Hgroup', 'Hr', 'Html', 'I', 'Iframe', 'Img', 'Input', 'Ins', 'Kbd', 'Label', 'Legend', 'Li',
    'Link', 'Main', 'Map', 'Mark', 'Menu', 'Meta', 'Meter', 'Nav', 'Noscript', 'Object', 'Ol', 'Optgroup', 'Option', 'Output',
    'P', 'Picture', 'PortalExperimental', 'Pre', 'Progress', 'Q', 'Rp', 'Rt', 'Ruby', 'S', 'Samp', 'Script', 'Search',
    'Section', 'Select', 'Slot', 'Small', 'Source', 'Span', 'Strong', 'Style', 'Sub', 'Summary', 'Sup', 'Table', 'Tbody',
    'Td', 'Template', 'Textarea', 'Tfoot', 'Th', 'Thead', 'Time', 'Title', 'Tr', 'Track', 'U', 'Ul', 'Var', 'Video', 'Wbr']
for o in _all_: _g[o] = partial(xt_hx, o.lower())

In [ ]:
#| export
def File(fname):
    "Use the unescaped text in file `fname` directly"
    return NotStr(Path(fname).read_text())

In [ ]:
#| hide
from fastcore.py2pyi import create_pyi

In [ ]:
#| hide
both_attrs = html_attrs+hx_attrs

create_pyi('../fasthtml/components.py', 'fasthtml')
create_pyi('../fasthtml/xtend.py', 'fasthtml')
with open('../fasthtml/components.pyi', 'a') as f:
    attrs_str = ', '.join(f'{t}:str|None=None' for t in both_attrs)
    f.write(f"\ndef xt_html(tag: str, *c, {attrs_str}, **kwargs): ...\n")
    f.write(f"def xt_hx(tag: str, *c, {attrs_str}, **kwargs): ...\n")
    for o in _all_:
        attrs = (['name'] if o.lower() in named else []) + both_attrs
        attrs_str = ', '.join(f'{t}:str|None=None' for t in attrs)
        f.write(f"def {o}(*c, {attrs_str}, **kwargs): ...\n")

For tags that have a `name` attribute, it will be set to the value of `id` if not provided explicitly:

In [ ]:
Form(Button(target_id='foo', id='btn'),
     hx_post='/', target_id='tgt', id='frm')

```xml
<form hx-post="/" hx-target="#tgt" id="frm" name="frm">
  <button hx-target="#foo" id="btn" name="btn"></button>
</form>

```

In [ ]:
#| export
def _fill_item(item, obj):
    if not isinstance(item,list): return item
    tag,cs,attr = item
    if isinstance(cs,tuple): cs = tuple(_fill_item(o, obj) for o in cs)
    name = attr.get('name', None)
    val = None if name is None else obj.get(name, None)
    if val is not None:
        if tag=='input':
            if attr.get('type', '') in ('checkbox','radio'):
                if val: attr['checked'] = '1'
                else: attr.pop('checked', '')
            else: attr['value'] = val
        if tag=='textarea': cs=(val,)
    return FT(tag,cs,attr)

In [ ]:
#| export
def fill_form(form:FT, obj)->FT:
    "Fills named items in `form` using attributes in `obj`"
    if not isinstance(obj,dict): obj = asdict(obj)
    return _fill_item(form, obj)

In [ ]:
@dataclass
class TodoItem:
    title:str; id:int; done:bool; details:str
                
todo = TodoItem(id=2, title="Profit", done=True, details="Details")
check = Label(Input(type="checkbox", cls="checkboxer", id="done", data_foo="bar"), "Done")
form = Form(Fieldset(Input(cls="char", id="title"), check, Input(type="hidden", id="id"), Textarea(id='details'), Button("Save")))
form = fill_form(form, todo)
form

```xml
<form>
  <fieldset>
    <input id="title" class="char" name="title" value="Profit"></input>
    <label>
      <input type="checkbox" data-foo="bar" id="done" class="checkboxer" name="done" checked="1"></input>
Done
    </label>
    <input type="hidden" id="id" name="id" value="2"></input>
    <textarea id="details" name="details">Details</textarea>
    <button>Save</button>
  </fieldset>
</form>

```

In [ ]:
#|export
def fill_dataclass(src, dest):
    "Modifies dataclass in-place and returns it"
    for nm,val in asdict(src).items(): setattr(dest, nm, val)
    return dest

In [ ]:
nt = TodoItem('', 0, False, '')
fill_dataclass(todo, nt)
nt

TodoItem(title='Profit', id=2, done=True, details='Details')

In [ ]:
#| exports
def find_inputs(e, tags='input', **kw):
    # Recursively find all elements in `e` with `tags` and attrs matching `kw`
    if not isinstance(e, (list,tuple)): return []
    inputs = []
    if isinstance(tags,str): tags = [tags]
    elif tags is None: tags = []
    cs = e
    if isinstance(e, list):
        tag,cs,attr = e
        if e[0] in tags and kw.items()<=e[2].items(): inputs.append(e)
    for o in cs: inputs += find_inputs(o, tags, **kw)
    return inputs

In [ ]:
find_inputs(form, id='title')

[['input',
  (),
  {'id': 'title', 'class': 'char', 'name': 'title', 'value': 'Profit'}]]

You can also use lxml for more sophisticated searching:

In [ ]:
elem = lx.fromstring(to_xml(form))
elem.xpath("//input[@id='title']/@value")

['Profit']

In [ ]:
#| export
def __getattr__(tag):
    if tag.startswith('_') or tag[0].islower(): raise AttributeError
    tag = tag.replace("_", "-")
    def _f(*c, target_id=None, **kwargs): return xt_hx(tag, *c, target_id=target_id, **kwargs)
    return _f

In [ ]:
#| export
_re_h2x_attr_key = re.compile(r'^[A-Za-z_-][\w-]*$')
def html2xt(html):
    """Convert HTML to an `ft` expression"""
    rev_map = {'class': 'cls', 'for': 'fr'}
    
    def _parse(elm, lvl=0, indent=4):
        if isinstance(elm, str): return repr(elm.strip()) if elm.strip() else ''
        if isinstance(elm, list): return '\n'.join(_parse(o, lvl) for o in elm)
        tag_name = elm.name.capitalize()
        if tag_name=='[document]': return _parse(list(elm.children), lvl)
        cts = elm.contents
        cs = [repr(c.strip()) if isinstance(c, str) else _parse(c, lvl+1)
              for c in cts if str(c).strip()]
        attrs = []
        for key, value in sorted(elm.attrs.items(), key=lambda x: x[0]=='class'):
            if isinstance(value,(tuple,list)): value = " ".join(value)
            key = rev_map.get(key, key)
            attrs.append(f'{key.replace("-", "_")}={value!r}' if _re_h2x_attr_key.match(key) else f'**{{{key!r}:{value!r}}}')
        spc = " "*lvl*indent
        onlychild = not cts or (len(cts)==1 and isinstance(cts[0],str))
        j = ', ' if onlychild else f',\n{spc}'
        inner = j.join(filter(None, cs+attrs))
        if onlychild: return f'{tag_name}({inner})'
        return f'{tag_name}(\n{spc}{inner}\n{" "*(lvl-1)*indent})'

    soup = BeautifulSoup(html.strip(), 'html.parser')
    for c in soup.find_all(string=risinstance(Comment)): c.extract()
    return _parse(soup, 1)

In [ ]:
h = to_xml(form)
hl_md(html2xt(h), 'python')

```python
Form(
    Fieldset(
        Input(id='title', name='title', value='Profit', cls='char'),
        Label(
            Input(type='checkbox', data_foo='bar', id='done', name='done', checked='1', cls='checkboxer'),
            'Done'
        ),
        Input(type='hidden', id='id', name='id', value='2'),
        Textarea('Details', id='details', name='details'),
        Button('Save')
    )
)
```

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()